In [62]:
import pandas as pd
import ast
import numpy as np
import math

In [63]:
df = pd.read_csv('attribution_allocation_student_data.csv')
df

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3
68120,False,referral,referral,referral,referral,direct,3
68121,False,display,referral,social,direct,NaN,3
68122,False,email,email,social,direct,NaN,3


In [64]:
input_file = pd.read_csv('channel_spend_student_data.csv', skiprows=1, names = ['tier', 'spend'])
input_file

,tier,spend
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


# calculate cost for each channel

In [67]:
input_file['spend']=input_file['spend'].apply(ast.literal_eval)

In [72]:
tier1_spend_dict = input_file.iloc[0,1]
tier2_spend_dict = input_file.iloc[1,1]
tier3_spend_dict = input_file.iloc[2,1]

In [74]:
tier1_spend = pd.DataFrame(list(tier1_spend_dict.items()),columns = ['channel','spend'])
tier2_spend = pd.DataFrame(list(tier2_spend_dict.items()),columns = ['channel','spend'])
tier3_spend = pd.DataFrame(list(tier3_spend_dict.items()),columns = ['channel','spend'])

In [75]:
tier1 = df[df['tier'] == 1]
tier2 = df[df['tier'] == 2]
tier3 = df[df['tier'] == 3]

# first interaction

In [94]:
convert1 = tier1[tier1['convert_TF'] == True]
convert2 = tier2[tier2['convert_TF'] == True]
convert3 = tier3[tier3['convert_TF'] == True]
print(convert1)
print(convert2)
print(convert3)

       convert_TF    touch1    touch2          touch3  touch4    touch5  tier
0            True  referral  referral        referral   email       NaN     1
1            True  referral   display         display   email       NaN     1
2            True     email   display        referral     NaN       NaN     1
3            True  referral  referral           email     NaN       NaN     1
4            True    social  referral             NaN     NaN       NaN     1
...           ...       ...       ...             ...     ...       ...   ...
4304         True  referral  referral        referral     NaN       NaN     1
4305         True  referral  referral          social  social  referral     1
11333        True  referral  referral  organic_search     NaN       NaN     1
11334        True  referral  referral  organic_search     NaN       NaN     1
11347        True  referral  referral             NaN     NaN       NaN     1

[2824 rows x 7 columns]
       convert_TF          touch1    to

In [93]:
first_touch1 = convert1['touch1'].value_counts().rename_axis('channel').reset_index(name='converts')
print(first_touch1)
first_touch2 = convert2['touch1'].value_counts().rename_axis('channel').reset_index(name='converts')
print(first_touch2)
first_touch3 = convert3['touch1'].value_counts().rename_axis('channel').reset_index(name='converts')
print(first_touch3)

          channel  converts
0        referral      1519
1          social       475
2         display       435
3           email       264
4     paid_search       129
5          direct         1
6  organic_search         1
          channel  converts
0        referral      2719
1          social       823
2         display       726
3           email       402
4     paid_search       211
5  organic_search         7
6          direct         4
          channel  converts
0        referral      3257
1          social       982
2         display       870
3           email       618
4     paid_search       328
5  organic_search         7
6          direct         1


In [95]:
first_interaction_cac1 = pd.merge(first_touch1, tier1_spend, how='inner', right_on=['channel'],left_on=['channel'])
print(first_interaction_cac1)
first_interaction_cac2 = pd.merge(first_touch2, tier2_spend, how='inner', right_on=['channel'],left_on=['channel'])
print(first_interaction_cac2)
first_interaction_cac3 = pd.merge(first_touch3, tier3_spend, how='inner', right_on=['channel'],left_on=['channel'])
print(first_interaction_cac3)

          channel  converts   spend
0        referral      1519  1000.0
1          social       475  1000.0
2         display       435  1000.0
3           email       264  1000.0
4     paid_search       129  1000.0
5          direct         1     0.0
6  organic_search         1     0.0
          channel  converts   spend
0        referral      2719  2000.0
1          social       823  2000.0
2         display       726  2000.0
3           email       402  2000.0
4     paid_search       211  2000.0
5  organic_search         7     0.0
6          direct         4     0.0
          channel  converts   spend
0        referral      3257  3000.0
1          social       982  3000.0
2         display       870  3000.0
3           email       618  3000.0
4     paid_search       328  3000.0
5  organic_search         7     0.0
6          direct         1     0.0


In [87]:
first_interaction_cac1['cac'] = first_interaction_cac1['spend']/first_interaction_cac1['converts']
first_interaction_cac1

,channel,converts,spend,cac
0,referral,1519,1000.0,0.658328
1,social,475,1000.0,2.105263
2,display,435,1000.0,2.298851
3,email,264,1000.0,3.787879
4,paid_search,129,1000.0,7.751938
5,direct,1,0.0,0.000000
6,organic_search,1,0.0,0.000000


In [96]:
first_interaction_cac2['cac'] = first_interaction_cac2['spend']/first_interaction_cac2['converts']
first_interaction_cac2

,channel,converts,spend,cac
0,referral,2719,2000.0,0.735565
1,social,823,2000.0,2.430134
2,display,726,2000.0,2.754821
3,email,402,2000.0,4.975124
4,paid_search,211,2000.0,9.478673
5,organic_search,7,0.0,0.000000
6,direct,4,0.0,0.000000


In [97]:
first_interaction_cac3['cac'] = first_interaction_cac3['spend']/first_interaction_cac3['converts']
first_interaction_cac3

,channel,converts,spend,cac
0,referral,3257,3000.0,0.921093
1,social,982,3000.0,3.054990
2,display,870,3000.0,3.448276
3,email,618,3000.0,4.854369
4,paid_search,328,3000.0,9.146341
5,organic_search,7,0.0,0.000000
6,direct,1,0.0,0.000000


In [99]:
first_interaction_cac = pd.DataFrame()
first_interaction_cac['channel'] = first_interaction_cac1['channel']
first_interaction_cac['converts'] = first_interaction_cac1['converts'] + first_interaction_cac2['converts'] + first_interaction_cac3['converts']
first_interaction_cac['spend'] = first_interaction_cac1['spend'] + first_interaction_cac2['spend'] + first_interaction_cac3['spend']
first_interaction_cac

,channel,converts,spend
0,referral,7495,6000.0
1,social,2280,6000.0
2,display,2031,6000.0
3,email,1284,6000.0
4,paid_search,668,6000.0
5,direct,15,0.0
6,organic_search,6,0.0


In [100]:
first_interaction_cac['cac'] = first_interaction_cac['spend']/first_interaction_cac['converts']
first_interaction_cac

,channel,converts,spend,cac
0,referral,7495,6000.0,0.800534
1,social,2280,6000.0,2.631579
2,display,2031,6000.0,2.954210
3,email,1284,6000.0,4.672897
4,paid_search,668,6000.0,8.982036
5,direct,15,0.0,0.000000
6,organic_search,6,0.0,0.000000
